In [1]:
# rough skeleton for our main ftn

from tdmclient import ClientAsync
from cv import *
from kalman import *
from motion_control import *
from utils import *
from path_planning import *
# and other imports

# repeated timer to get data from thymio(velocity, acceleration, proximity sensor)
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

def get_data():
    {"ground":list(node["prox.ground.reflected"]), 
                        "sensor":list(node["prox.ground.reflected"]),
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"],
                        "acc":list(node["acc"])}

In [ ]:
# connect to thymio
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

# connect to camera, initialize map, compute global path

# main loop
while (not goal_reached):
    #get data from camera(thymio_found, x, y, theta)
    #get data from thymio(proximity, acceleration, motor speed)

    #if kidnapping detected(from acceleration), compute global path again, initialize ekf again, continue;
    #updated x,y,theta,v, w = ekf.update (x,y,theta from cv, motor speed from thymio data, thymio_found))
    #is goal reached? check with updated x,y and goal position from cv
    #ul, ur = motion control(updated x,y,theta, ) -> global path following / local avoidance is handled here
    #node.send_set_variables(motor(ul, ur))
    #await client.sleep(dt)
